In [13]:
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session
import pandas as pd 
sql = """
select * from tt_h.parsed_rail_journeys
"""
df = pd.read_sql(sql, conn)

In [14]:
df_pf = df[df["tfl_dest"] == "SW1H9AJ"]
df_cw = df[df["tfl_dest"] == "E145HP"]

In [15]:
all_stations_data = df_pf.merge(df_cw, how='left', on=["nlc", "icscode", "station_name", "tfl_message", "depart_lat", "depart_lng"], suffixes=("_pf", "_cw"))

In [ ]:
# Now merge on drivetimes

In [45]:
sql = """
select icscode, dest_pc, drive_minutes, drive_miles from tt_h.stations_drivetime
"""
dt = pd.read_sql(sql, conn)
pt = dt.pivot_table(index=["icscode"], columns="dest_pc")
pt.columns = ['_'.join(col).strip().lower() for col in pt.columns.values]
pt = pt.reset_index()

In [47]:
all_stations_data_2 = all_stations_data.merge(pt, on="icscode", how="left")

In [54]:
all_stations_data_2.columns

Index(['nlc', 'icscode', 'station_name', 'tfl_message', 'tfl_dest_pf', 'lat',
       'lng', 'natrail_journey_summary_pf', 'natrail_train_changes_pf',
       'final_arrival_pf', 'natrail_journey_minutes_pf', 'arrive_lat_pf',
       'arrive_lng_pf', 'cycle_minutes_pf', 'cycle_miles_pf',
       'total_journeytime_pf', 'id_pf', 'tfl_dest_cw',
       'natrail_journey_summary_cw', 'natrail_train_changes_cw',
       'final_arrival_cw', 'natrail_journey_minutes_cw', 'arrive_lat_cw',
       'arrive_lng_cw', 'cycle_minutes_cw', 'cycle_miles_cw',
       'total_journeytime_cw', 'id_cw', 'drive_miles_cb113qy',
       'drive_miles_ox145fp', 'drive_minutes_cb113qy',
       'drive_minutes_ox145fp'],
      dtype='object')

In [65]:
import numpy as np
all_stations_data_2["parents_drivetime_diff"] =  np.abs(all_stations_data_2["drive_minutes_ox145fp"] - all_stations_data_2["drive_minutes_cb113qy"])
all_stations_data_2["parents_drivetime_total"] =  all_stations_data_2["drive_minutes_ox145fp"] + all_stations_data_2["drive_minutes_cb113qy"]
all_stations_data_2["parents_drivetime_score"] =  all_stations_data_2["parents_drivetime_total"] + all_stations_data_2["parents_drivetime_diff"]

In [66]:
all_stations_data_2.rename(columns={"depart_lat":"lat", "depart_lng":"lng"}, inplace=True)
all_stations_data_2.to_csv("interactive_maps_template/data/stations_maps_data.csv", encoding='utf-8', index=False)